# Exploratory Data Analysis 

### Questions

1. What is the distribution of ages within the dataset? Is there an age range that is more prevalent?
2. What is the distribution of genders?
3. Corrupted data/missing values

In [57]:
import os
import scipy.io as sio
from pprint import pprint


imdb_path = "../dataset/imdb_crop"
wiki_path = "../dataset/wiki_crop"

for folder_name in os.listdir(imdb_path):
    folder_path = os.path.join(imdb_path, folder_name)
    
    if os.path.isdir(folder_path):
        # Looping through each image in the current folder
        for file_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, file_name)
            # todo process image?
        
    else: # This is the .mat file
        imdb_mat = sio.loadmat(folder_path)
        
        imdb_dict = {}
        
    
        num_images = len(imdb_mat["imdb"][0][0][0][0])
        imdb_mat_content = imdb_mat["imdb"][0][0]
        
        for i in range(num_images):
            dob = imdb_mat_content[0][0][i]  # date of birth (Matlab serial date number)
            photo_taken = imdb_mat_content[1][0][i]  # year when the photo was taken
            full_path = imdb_mat_content[2][0][i]  # path to file
            gender = imdb_mat_content[3][0][i]  # 0 for female and 1 for male, NaN if unknown
            name = imdb_mat_content[4][0][i]  # name of the celebrity
            face_location = imdb_mat_content[5][0][i]  # location of the face
            face_score = imdb_mat_content[6][0][i]  # detector score (the higher the better). Inf implies that no face was found in the image and the face_location then just returns the entire image
            second_face_score = imdb_mat_content[7][0][i]  # detector score of the face with the second highest score. This is useful to ignore images with more than one face. second_face_score is NaN if no second face was detected.
            
            imdb_dict[str(full_path)] = {
                "dob": int(dob) / 365
            }
        print(imdb_dict)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [67]:
import os
import numpy as np
import scipy.io as sio
import pandas as pd
from pprint import pprint

In [64]:
imdb_mat_path = "../dataset/imdb_crop/imdb.mat"
imdb_mat = sio.loadmat(imdb_mat_path)

imdb_dict = {}
        
    
num_images = len(imdb_mat["imdb"][0][0][0][0])
imdb_mat_content = imdb_mat["imdb"][0][0]

data = []


dob = imdb_mat_content[0][0]  # date of birth (Matlab serial date number)
photo_taken = imdb_mat_content[1][0]  # year when the photo was taken
full_path = imdb_mat_content[2][0]  # path to file
gender = imdb_mat_content[3][0]  # 0 for female and 1 for male, NaN if unknown
name = imdb_mat_content[4][0]  # name of the celebrity
face_location = imdb_mat_content[5][0]  # location of the face
face_score = imdb_mat_content[6][0]  # detector score (the higher the better). Inf implies that no face was found in the image and the face_location then just returns the entire image
second_face_score = imdb_mat_content[7][0]  # detector score of the face with the second highest score. This is useful to ignore images with more than one face. second_face_score is NaN if no second face was detected.

print(type(dob))
    
df = pd.DataFrame({
    'DOB': dob, 
    'photo_taken': photo_taken, 
    'full_path': full_path, 
    'gender':gender, 
    'name':name,
    'face_score':face_score, 
    })
df['name'] = df['name'].apply(lambda x: x[0])
df['full_path'] = df['full_path'].apply(lambda x: x[0])
df


<class 'numpy.ndarray'>


,DOB,photo_taken,full_path,gender,name,face_score
0,693726,1968,01/nm0000001_rm124825600_1899-5-10_1968.jpg,1.0,Fred Astaire,1.459693
1,693726,1970,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,1.0,Fred Astaire,2.543198
2,693726,1968,01/nm0000001_rm577153792_1899-5-10_1968.jpg,1.0,Fred Astaire,3.455579
3,693726,1968,01/nm0000001_rm946909184_1899-5-10_1968.jpg,1.0,Fred Astaire,1.872117
4,693726,1968,01/nm0000001_rm980463616_1899-5-10_1968.jpg,1.0,Fred Astaire,1.158766
...,...,...,...,...,...,...
460718,726831,2011,08/nm3994408_rm761245696_1989-12-29_2011.jpg,0.0,Jane Levy,3.845884
460719,726831,2011,08/nm3994408_rm784182528_1989-12-29_2011.jpg,0.0,Jane Levy,-inf
460720,726831,2011,08/nm3994408_rm926592512_1989-12-29_2011.jpg,0.0,Jane Levy,-inf
460721,726831,2011,08/nm3994408_rm943369728_1989-12-29_2011.jpg,0.0,Jane Levy,4.450725


### Data Quality

In [38]:
duplicateRows = df[df.duplicated()]
duplicateRows

,DOB,photo_taken,full_path,gender,name,face_score,second_face_score


In [69]:
#  Removing genders with the value NaN, meaning that gender is unknown
df = df[(df["gender"] == 1) | 
        (df["gender"] == 0)]
df

,DOB,photo_taken,full_path,gender,name,face_score
0,693726,1968,01/nm0000001_rm124825600_1899-5-10_1968.jpg,1.0,Fred Astaire,1.459693
1,693726,1970,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,1.0,Fred Astaire,2.543198
2,693726,1968,01/nm0000001_rm577153792_1899-5-10_1968.jpg,1.0,Fred Astaire,3.455579
3,693726,1968,01/nm0000001_rm946909184_1899-5-10_1968.jpg,1.0,Fred Astaire,1.872117
4,693726,1968,01/nm0000001_rm980463616_1899-5-10_1968.jpg,1.0,Fred Astaire,1.158766
...,...,...,...,...,...,...
460718,726831,2011,08/nm3994408_rm761245696_1989-12-29_2011.jpg,0.0,Jane Levy,3.845884
460719,726831,2011,08/nm3994408_rm784182528_1989-12-29_2011.jpg,0.0,Jane Levy,-inf
460720,726831,2011,08/nm3994408_rm926592512_1989-12-29_2011.jpg,0.0,Jane Levy,-inf
460721,726831,2011,08/nm3994408_rm943369728_1989-12-29_2011.jpg,0.0,Jane Levy,4.450725


In [70]:
#  Removing face scores with the value -inf, meaning that there was no face detected

df = df[df['face_score'] != -np.inf]
df

,DOB,photo_taken,full_path,gender,name,face_score
0,693726,1968,01/nm0000001_rm124825600_1899-5-10_1968.jpg,1.0,Fred Astaire,1.459693
1,693726,1970,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,1.0,Fred Astaire,2.543198
2,693726,1968,01/nm0000001_rm577153792_1899-5-10_1968.jpg,1.0,Fred Astaire,3.455579
3,693726,1968,01/nm0000001_rm946909184_1899-5-10_1968.jpg,1.0,Fred Astaire,1.872117
4,693726,1968,01/nm0000001_rm980463616_1899-5-10_1968.jpg,1.0,Fred Astaire,1.158766
...,...,...,...,...,...,...
460715,726831,2011,08/nm3994408_rm73386752_1989-12-29_2011.jpg,0.0,Jane Levy,4.801987
460716,726831,2011,08/nm3994408_rm744468480_1989-12-29_2011.jpg,0.0,Jane Levy,2.370091
460718,726831,2011,08/nm3994408_rm761245696_1989-12-29_2011.jpg,0.0,Jane Levy,3.845884
460721,726831,2011,08/nm3994408_rm943369728_1989-12-29_2011.jpg,0.0,Jane Levy,4.450725
